In [ ]:
import os
import numpy as np
import pandas as pd
os.chdir('/content')
!mkdir /root/.kaggle
!echo '{"username":"reedbalentine","key":"0a77017a301886b3c06793f4c21c30fc"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts
if not os.path.exists("/content/NPR_Data"):
    os.makedirs("/content/NPR_Data")


 99% 293M/295M [00:14<00:00, 18.8MB/s]
100% 295M/295M [00:14<00:00, 21.8MB/s]


In [ ]:
!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data

Archive:  interview-npr-media-dialog-transcripts.zip
  inflating: NPR_Data/episodes.csv   
  inflating: NPR_Data/headlines.csv  
  inflating: NPR_Data/host-map.json  
  inflating: NPR_Data/host_id.json   
  inflating: NPR_Data/splits-ns2.json  
  inflating: NPR_Data/utterances-2sp.csv  
  inflating: NPR_Data/utterances.csv  


In [ ]:
os.chdir('/content/NPR_Data')

In [ ]:
df = pd.read_csv('utterances.csv')
df['speaker'] = df['speaker'].str.upper()
df['speaker'] = df['speaker'].str.split(', H', expand=True)[0]

In [ ]:
df = df[df['speaker'] != '_NO_SPEAKER']

In [ ]:
df['utterance_len'] = df['utterance'].str.len()

In [ ]:
df = df.dropna()
host_id_df = pd.read_json('host_id.json', orient='index')

In [ ]:
host_id_df = pd.read_json('host_id.json', orient='index')
host_id_df.reset_index(inplace=True)
host_id_df = host_id_df.rename(columns = {'index':'speaker'})

In [ ]:
host_id_df['speaker'] = host_id_df['speaker'].astype('string')
host_id_df['speaker'] = host_id_df['speaker'].str.upper()
host_id_df.rename(columns={0:'host_id'}, inplace=True)

In [ ]:
mergedf = df.merge(host_id_df, how='left', on='speaker')

In [ ]:
mergedf['host_id'] = mergedf['host_id'].fillna(-1)

In [ ]:
PreProcess_df = mergedf[mergedf['host_id'] != -1]

In [ ]:
PreProcess_df = PreProcess_df.loc[PreProcess_df['utterance_len'] >= 100]

In [ ]:
top_speakers = PreProcess_df.groupby(['speaker']).size().loc[PreProcess_df.groupby(['speaker']).size() > 10000]
df = pd.DataFrame(PreProcess_df.loc[PreProcess_df['speaker'].isin(top_speakers.index.values)])
df = df.reset_index(drop=True)

In [ ]:
df['utterance'] = df['utterance'].str.lower()

In [ ]:
df['unidentified'] = df['utterance'].str.startswith('unidentified')
df = df[df['unidentified'] == False]

,episode,episode_order,speaker,utterance,utterance_len,host_id,unidentified
0,57264,15,NEAL CONAN,and so the guy who's in back has an easier tim...,276.0,7.0,False
1,57264,17,NEAL CONAN,"we're talking with loren mooney, the editor-in...",301.0,7.0,False
2,57264,28,NEAL CONAN,"bye-bye. there is also, as we mentioned, the t...",330.0,7.0,False
3,57264,38,NEAL CONAN,"one of interesting things, marshall, that i fi...",254.0,7.0,False
4,57264,61,NEAL CONAN,and here's some emails that we have. this from...,535.0,7.0,False
...,...,...,...,...,...,...,...
51762,57264,2,NEAL CONAN,"but not unlike soccer, the popularity of bicyc...",382.0,7.0,False
51763,64113,40,NEAL CONAN,and if you'd like to talk with russell peters ...,754.0,7.0,False
51764,64113,139,NEAL CONAN,we're going to cry a river with russell peters...,265.0,7.0,False
51765,64113,141,NEAL CONAN,this is talk of the nation. i'm neal conan in ...,323.0,7.0,False


In [ ]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def preprocess_text(text):
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]

    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

In [ ]:
utterance_list = df['utterance'].to_list()
final_list = []
complete_list = []
for utterance in utterance_list:
    processed = preprocess_text(utterance)
    final_list.append(processed)
complete_list.extend(final_list)
df['processed_utterance'] = complete_list

KeyboardInterrupt: ignored

In [ ]:
df['processed_utterance'] = df['processed_utterance'].apply(lambda x: ' '.join(x))

In [ ]:
df


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
X = df.processed_utterance
y = df.speaker
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.steps

param_grid = {}
param_grid["tfidfvectorizer__max_features"] = [500, 1000, 15000]
param_grid["tfidfvectorizer__ngram_range"] = [(1,1), (1,2), (2,2)]
param_grid["tfidfvectorizer__stop_words"] = ["english", None]
param_grid["tfidfvectorizer__strip_accents"] = ["ascii", "unicode", None]
param_grid["tfidfvectorizer__analyzer"] = ["word", "char"]
param_grid["tfidfvectorizer__binary"] = [True, False]
param_grid["tfidfvectorizer__norm"] = ["l1", "l2", None]
param_grid["tfidfvectorizer__use_idf"] = [True, False]
param_grid["tfidfvectorizer__smooth_idf"] = [True, False]
param_grid["tfidfvectorizer__sublinear_tf"] = [True, False]


grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', verbose = 3, n_jobs = -1)
grid.get_params().keys()

In [ ]:
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
vector = TfidfVectorizer(analyzer='word', stop_words='english', max_features = 850, ngram_range=(1, 1), 
                       binary=False, norm=None, smooth_idf=True, strip_accents=None,
                       sublinear_tf=True, use_idf=False)

df_transformed = vector.fit_transform(X_train)

In [ ]:
multi_nb_model = MultinomialNB()
multi_nb_model.fit(df_transformed, y_train)
print ("Model accuracy within dataset: ", multi_nb_model.score(df_transformed, y_train))

In [ ]:
test_text = ["So for a top competitor like Lance to try to"]
test_text_transform = vector.transform(test_text)

print (multi_nb_model.predict(test_text_transform)," most likely said it.")